# Visszakeresés alapú magyar nyelvű chatbot program:



1.Importálom a szükséges könyvtárakat:

In [ ]:
#!pip install spacy
#!pip install scikit-learn
import spacy #én ezt használom, hogy magyar nyelven listázzam a magukban értelmetlen szavakat.
import string # tud stringekkel dolgozni
import json # tud json fájllal dolgozni (betölteni)
from sklearn.feature_extraction.text import CountVectorizer # szöveg numerikus vektorrá alakításhoz
from sklearn.model_selection import train_test_split # adatok képzési és tesztelése készletekre osztásához
from sklearn.linear_model import LogisticRegression #logiszikai regressziós modellhez
from sklearn.preprocessing import LabelEncoder # címkék numerikus ábrázolásához
import random # kiválasztáshoz
from sklearn.ensemble import RandomForestClassifier #döntési fák generálásához
from sklearn.feature_extraction.text import TfidfVectorizer #szöveges dokumentum -> TF-IDF Mátrix
from sklearn.svm import SVC # Support Vector Machines osztályozó

2.Létrehozom a stopwords listát:


In [ ]:
nlp = spacy.blank("xx") # Általános tokenizátor, nyelvspecifikus modellek nélkül, mert a magyar verzió nem működik.

hungarian_stopwords = set([
    'a', 'az', 'abba', 'abban', 'abból', 'addig', 'ahhoz', 'ahogy', 'ahol', 'aki', 'akik', 'akkor', 'akár', 'alapján',
    'alatt', 'alatta', 'alá', 'alól', 'amely', 'amelyek', 'amelyekben', 'amelyeket', 'amelyet', 'amelynek', 'ami',
    'amikor', 'amit', 'amolyan', 'amíg', 'annak', 'arra', 'arról', 'attól', 'az', 'aznap', 'azok', 'azokat', 'azokba',
    'azokban', 'azokból', 'azokhoz', 'azokig', 'azokkal', 'azokká', 'azoknak', 'azoknál', 'azokon', 'azokra', 'azokról',
    'azoktól', 'azon', 'azonban', 'azonnal', 'azt', 'aztán', 'azután', 'azzal', 'azért', 'be', 'belé', 'beléd', 'beléjük',
    'belém', 'belénk', 'belétek', 'belül', 'benne', 'benned', 'bennem', 'bennetek', 'bennük', 'bennünk', 'bár', 'bárcsak',
    'bármilyen', 'csak', 'de', 'egy', 'egyből', 'egyedül', 'egyelőre', 'egyes', 'egyet', 'egyetlen', 'egyik', 'egymás',
    'egyre', 'egyszer', 'egyéb', 'együtt', 'ehhez', 'ekkor', 'el', 'eleinte', 'ellen', 'ellenes', 'elleni', 'ellenére',
    'elmondta', 'első', 'elsők', 'elsősorban', 'elé', 'elég', 'elő', 'előbb', 'elől', 'előle', 'előled', 'előlem',
    'előletek', 'előlük', 'először', 'előtt', 'előtte', 'előtted', 'előttem', 'előttetek', 'előttük', 'előttünk', 'emilyen',
    'ennek', 'ennyi', 'ennél', 'enyém', 'erre', 'erről', 'esetben', 'ettől', 'ez', 'ezek', 'ezekbe', 'ezekben', 'ezekből',
    'ezeken', 'ezeket', 'ezekhez', 'ezekig', 'ezekkel', 'ezekké', 'ezeknek', 'ezeknél', 'ezekre', 'ezekről', 'ezektől',
    'ezért', 'fel', 'felé', 'fölé', 'ha', 'halló', 'hanem', 'hiszen', 'hiába', 'hogy', 'hogyan', 'hol', 'holnap', 'honnan',
    'hova', 'hozzá', 'hozzád', 'hozzájuk', 'hozzám', 'hozzánk', 'hozzátok', 'igen', 'ill', 'illetve', 'ilyen', 'inkább',
    'is', 'ismét', 'itt', 'jó', 'jól', 'jobban', 'kell', 'kellene', 'kellett', 'keressünk', 'keresztül', 'kevés', 'ki',
    'kicsit', 'kik', 'kilenc', 'kin', 'kinek', 'kinél', 'kire', 'kiről', 'kit', 'kitől', 'kivel', 'körül', 'közben', 'között',
    'közül', 'külön', 'különben', 'lassan', 'le', 'legalább', 'legyen', 'lehet', 'lehetetlen', 'lehetett', 'lenne', 'lenni',
    'lett', 'lévő', 'ma', 'maga', 'magad', 'magam', 'magatokat', 'magukat', 'magunkat', 'magát', 'mai', 'majd', 'majdnem',
    'meg', 'megint', 'megvan', 'mellett', 'mellette', 'melletted', 'mellettem', 'mellettetek', 'mellettük', 'mellettünk',
    'mely', 'melyek', 'mennyire', 'mennyi', 'mert', 'mi', 'miatt', 'miatta', 'miben', 'miből', 'micsoda', 'miért', 'mikor',
    'mikorra', 'mind', 'mindegy', 'minden', 'mindenki', 'mindig', 'minél', 'mint', 'mintha', 'mivel', 'most', 'nagy',
    'nagyon', 'ne', 'nélkül', 'nincs', 'nyolc', 'nyolcadik', 'olyan', 'on', 'ott', 'pedig', 'persze', 'sem', 'semmi',
    'sok', 'sokat', 'sokkal', 'során', 'számára', 'szemben', 'szerint', 'szinte', 'talán', 'tartozik', 'tovább', 'továbbá',
    'több', 'úgy', 'ugyanis', 'új', 'újra', 'után', 'utána', 'vagy', 'valami', 'valamikor', 'valamint', 'való', 'van',
    'vannak', 'volt', 'voltam', 'voltak', 'voltunk', 'vele', 'viszont', 'volna', 'alá', 'bele', 'felé', 'hát', 'ide', 'oda',
    'össze', 'szét', 'vissza', 'rá', 'át', 'igen', 'nem', 'én', 'te', 'ő', 'mi', 'ti', 'ők', 'ön', 'magunk', 'akár', 'annak',
    'arra', 'arról', 'azon', 'azt', 'azzal', 'azonban', 'e', 'eddig', 'egyéb', 'egyszer', 'első', 'éppen', 'ez', 'ezen',
    'ezért', 'ha', 'hanem', 'hiszen', 'így', 'ilyenkor', 'ismét', 'itt', 'jó', 'jól', 'keresztül', 'kívül', 'lehet', 'magát',
    'magukat', 'mellett', 'mert', 'mivel', 'most', 'nagyon', 'ne', 'nincs', 'olyan', 'ott', 'pedig', 'sem', 'sok', 'szerint',
    'talán', 'tovább', 'úgy', 'új', 'után', 'vagy', 'valaki', 'valami', 'vannak', 'volt', 'voltam', 'voltak', 'voltunk',
    'velem', 'velük', 'lesz', 'lett', 'magunk', 'sajnos', 'külön', 'pont', 'esetleg', 'mind', 'kis', 'szó', 'akit', 'amit',
    'valamit', 'valamiért', 'magam', 'mindneki', 'mindenkinek', 'mindenkit',
    'evvel', 'tőle', 'tőlem', 'tőlünk', 'tőletek', 'tőlük', 'és'
])


3.A magyar szöveg tokenizálása:



In [ ]:
def preprocess_hungarian_text(text): #tokenizálás. Tokenizáló nyelvspecifikus modell nélkül szavakká alakítja a szöveget.
    doc = nlp(text.lower())
    cleaned_words = [token.text for token in doc if token.text not in hungarian_stopwords and token.text not in string.punctuation] # Kiszűri az egyéni stopszavakat és írásjeleket.

    return cleaned_words

sentence = "Helló! Hogy vagy ma?"
cleaned_text = preprocess_hungarian_text(sentence)
print(cleaned_text)

['helló']


4. "Intents" json fájl létrehozása, jegyzetfüzethez adása és betöltése:

In [ ]:
with open('intents.json', encoding='utf-8') as file:
    intents = json.load(file)

5.Szöveg vektorizálása:

5.1. Korpusz (gyűjtemény) látrehozása:

In [ ]:
corpus = [] #patterns, bemeneti minták
labels = [] #tag, címke

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Minden minta tokineizálása és előfeldolgozása.
        corpus.append(' '.join(preprocess_hungarian_text(pattern))) # A szavak összeillesztése mondattá és hozzáfűzés a gyűjteményhez
        labels.append(intent['tag']) #címke hozzáfűzése

5.2. Vektorizáljuk a korpuszt a CountVectorizer-rel. Korpus -> BoW mátrix (átalakítás)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2)) #inicializálás

X = vectorizer.fit_transform(corpus).toarray() # Vektorizálót ráilleszti a korpuszra és átalakítja  a korpuszt numerikus formába

encoder = LabelEncoder()
Y = encoder.fit_transform(labels) # Konvertálja a címkéket számokká


print("Szókincs:", vectorizer.get_feature_names_out())
print("Minták numerikus ábrázolása:", X)

Szókincs: ['adat' 'adatokat' 'adatokhoz' 'add' 'adhatnál' 'adhatnál nekem'
 'adhatnál részleteket' 'adni' 'adni nekem' 'adni tanácsot' 'ajánlani'
 'ajánlasz' 'ajánlasz rendelésre' 'akartam' 'akartam mondani' 'alakul'
 'alakul idő' 'alakul időjárás' 'alakul nálad' 'alakulnak'
 'alakulnak árak' 'alig' 'alig bírom' 'alig várom' 'alkalmas'
 'alkalmas időpont' 'alkalom' 'asztalt' 'asztalt foglalni'
 'asztalt nálatok' 'benyújtani' 'beszélgetést' 'beszélhetünk'
 'beszélhetünk még' 'beszéljünk' 'beszéljünk még' 'beszélni' 'beszéltünk'
 'beszélünk' 'biztos' 'biztos információ' 'biztosan' 'biztosra'
 'biztosra vehetjük' 'bocsánat' 'bocsánat akartam'
 'bocsánat kellemetlenségért' 'bocsánatot' 'bocsánatot kérek' 'bocsátani'
 'bocsátani egymásnak' 'bírom' 'bírom kivárni' 'bízz' 'bízz magadban'
 'büszke' 'büszke vagyok' 'bővebben' 'bővebben beszélni' 'csinálni'
 'csinálsz' 'csinálsz mostanában' 'csináltál' 'csodálatos' 'célomat'
 'dolgok' 'díja' 'ebben' 'egyetértek' 'egyetértek veled' 'egyeztethetné

6.Modell elkészítése:

6.1. Felosztjuk az adatokat:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=100) # 80% edzésre és 20% kiértékelésre bontottam.
# edzéshez használt minták: X_train
# kiértékelési minták: X_test
# edzéshez használt címkék: Y_train
# kiértékelési címkék: Y_test

6.2. Be kell tanítani az adatokat:

In [ ]:
model = SVC()
model.fit(X_train, Y_train)
model = RandomForestClassifier(n_estimators=300, min_samples_split=20) # modell létrehozása
model.fit(X_train, Y_train) # modell tanítása

RandomForestClassifier(min_samples_split=20, n_estimators=300)

6.3. Értékelni kell a modellt:

In [ ]:
accuracy = model.score(X_test, Y_test)
print(f'Model accuracy: {accuracy * 100:.2f}%')

Model accuracy: 51.32%


7.Megjósolja az új felhasználó bevitelének szándékát (melyik kategóriház tartozik):

In [ ]:
def predict_intent(user_input):
    # Felhasználói bemenet vektorizálása
    user_input_cleaned = ' '.join(preprocess_hungarian_text(user_input))
    user_input_vector = vectorizer.transform([user_input_cleaned]).toarray()

    # Megjósolja a címkéket (y)
    intent_label_encoded = model.predict(user_input_vector)[0]

    # Numerikus címkét visszaalakítja az eredetire
    intent_tag = encoder.inverse_transform([intent_label_encoded])[0]

    return intent_tag

user_input = "Mondj egy viccet"
predicted_intent = predict_intent(user_input)
print(f'A chatbot válasza a következő szándékon alapul: {predicted_intent}')

A chatbot válasza a következő szándékon alapul: vicc


8. Válaszok lekérése az intents file-ból (véletlenszerű választ kérhetünk le az intent.json fájl megfelelő listájáról.):

In [ ]:
def get_response(intent_tag):
    #print(f"Megjósolt címke: {intent_tag}")

    for intent in intents['intents']:
        #print(f"Címkék keresése: {intent['tag']}")

        if intent['tag'] == intent_tag:
           # print(f"Találat a címkék közül: {intent['tag']}")
            return random.choice(intent['responses'])

    # Ha nem található egyezés
    return "Bocsánat, de nem értettem"

9. Chatbot tesztelése:

In [ ]:
intent_map = {i: tag for i, tag in enumerate(encoder.classes_)}
while True:
    user_input = input("Te: ")
    if user_input.lower() == "kilépés":
        break

    # Felhasználói bevitel előfeldolgozása
    cleaned_input = ' '.join(preprocess_hungarian_text(user_input))

    # Szándék megjósolása
    predicted_intent = model.predict(vectorizer.transform([cleaned_input]))[0]

    # Válasz kiírása
    response = get_response(intent_map[predicted_intent])
    print(f"Bot: {response}")

Te: Szia!
Bot: Helló! Mi járatban vagy?
Te: Tudnál segíteni?
Bot: Természetesen! Hogyan segíthetek?
Te: Milyen lesz az időjárás holnap?
Bot: Ma szél várható, ne felejts el kabátot hozni!
Te: Köszönöm a segítséget
Bot: Bármikor, örülök, hogy segíthettem!
Te: VISZLÁT
Bot: Jó éjszakát! Szép álmokat!
Te: kilépés
